# 03 Datos abiertos sobre COVID-19

Estudiaremos los datos abiertos sobre la [vacunación COVID-19 en México](https://datos.gob.mx/busca/dataset/informacion-referente-a-casos-covid-19-en-mexico). Para saber las columnas baje el [Diccionario de Datos](https://www.gob.mx/salud/documentos/datos-abiertos-152127).

1. Descargue **datos_abiertos_covid19.zip** en el vínculo anterior.
2. Descomprima el archivo y póngalo en el mismo directorio donde está este _notebook_.
3. Cambie el nombre del archivo en `archivo_csv = Path("______COVID19MEXICO.csv")`
4. En la siguiente celda debe decir "Ya veo el archivo".

In [ ]:
# Pathlib es para usar archivos de nuestro sistema operativo
from pathlib import Path

archivo_csv = Path("211008COVID19MEXICO.csv")
if archivo_csv.exists():
    print("Ya veo el archivo", archivo_csv.name)
else:
    raise Exception("No lo encuentro")

In [ ]:
# Pandas es excelente para analizar datos
import pandas as pd

In [ ]:
# Cargar el archivo. NOTA: Se tarda un buen tiempo por su tamaño, si falla agregue low_memory=False
covid19 = pd.read_csv(archivo_csv, low_memory=False)

In [ ]:
# Dar un vistazo a sus primeros renglones
covid19.head()

In [ ]:
# Mostrar las columnas
covid19.columns

In [ ]:
# Coahuila de Zaragoza es la entidad 5
# RESULTADO_LAB 1 es POSITIVO A SARS-COV-2
entidad = covid19[(covid19.ENTIDAD_RES == 5) & (covid19.RESULTADO_LAB == 1)]
len(entidad)

In [ ]:
# Elegimos unas columnas y las redefinimos como categorias para se cuantifiquen
entidad = entidad[["SEXO", "ENTIDAD_RES", "MUNICIPIO_RES", "FECHA_DEF", "INTUBADO", "EDAD"]]
entidad.SEXO = entidad.SEXO.astype('category')
entidad.ENTIDAD_RES = entidad.ENTIDAD_RES.astype('category')
entidad.MUNICIPIO_RES = entidad.MUNICIPIO_RES.astype('category')
entidad.INTUBADO = entidad.INTUBADO.astype('category')
entidad.info()

In [ ]:
# Los sexos
entidad.SEXO.cat.categories

In [ ]:
# Reemplazar los sexos de numero a texto
entidad["SEXO"] = entidad["SEXO"].replace(to_replace=[1,2], value=["MUJER","HOMBRE"])

In [ ]:
# Los municipios
entidad.MUNICIPIO_RES.cat.categories

In [ ]:
# Cargar listados con las claves de los municipios de Coahuila
import csv
claves = []
nombres = []
with open("municipios_coahuila.csv", encoding="utf8") as puntero:
    renglones = csv.DictReader(puntero)
    for renglon in renglones:
        claves.append(int(renglon["CLAVE_MUNICIPIO"]))
        nombres.append(renglon["MUNICIPIO"])
print(claves)
print(nombres)

In [ ]:
# Reeplazar esas claves por los nombres de los municipios de Coahuila
entidad["MUNICIPIO_RES"] = entidad["MUNICIPIO_RES"].replace(to_replace=claves, value=nombres)
entidad.head()

In [ ]:
# Las fechas de defuncion
entidad.FECHA_DEF.unique()

In [ ]:
# Pivot table por sexo
entidad.pivot_table(
    index=["MUNICIPIO_RES"], 
    columns=pd.Grouper(key="SEXO"), 
    values="ENTIDAD_RES", 
    aggfunc='count',
)

In [ ]:
# Pivot table por edad
entidad.pivot_table(
    index=["MUNICIPIO_RES"], 
    columns=pd.Grouper(key="EDAD"), 
    values="ENTIDAD_RES", 
    aggfunc='count',
)